In [10]:
!pip install transformers

In [11]:
from google.colab import drive

current_directory = '/content/drive/My Drive/FSem/'
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Removal filters

In [12]:
def instead_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "instead":
            if i + 1 < len(tokens) and tokens[i + 1] == "of":
                pass
            else:
                tokens.pop(i)
                changed = True
                i = i - 1
        i = i + 1
    return changed

def ever_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "ever":
            if i > 0 and tokens[i - 1] == "than":
                pass
            else:
                tokens.pop(i)
                changed = True
                i = i - 1
        i = i + 1
    return changed

def anymore_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "anymore":
            tokens.pop(i)
            changed = True
            i = i - 1
        i = i + 1
    return changed

def too_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "too" and i + 1 < len(tokens) and tokens[i + 1] == ".":
            tokens.pop(i)
            i = i - 1
            if i >= 0 and tokens[i] == ",":
                tokens.pop(i)
                i = i - 1
            changed = True
        i = i + 1
    return changed

def eventually_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "eventually":
            tokens.pop(i)
            if i < len(tokens) and tokens[i] == ",":
                tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def immediately_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "immediately":
            tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def anyway_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "anyway":
            tokens.pop(i)
            i = i - 1
            changed = True
        i = i + 1
    return changed

def soon_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "soon":
            if i > 0 and tokens[i - 1] == "as":
                pass
            else:
                tokens.pop(i)
                if i < len(tokens) and tokens[i] == ",":
                    tokens.pop(i)
                i = i - 1
                changed = True
        i = i + 1
    return changed

def later_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "later":
            if i > 3 and tokens[i - 4] == "two" and tokens[i - 3] == "mile" and tokens[i - 2] == "##s" and tokens[i - 1] == "half":
                pass
            elif i > 1 and tokens[i - 2] == "month" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "week" and tokens[i - 1] == "##s":
                pass
            elif i > 1 and tokens[i - 2] == "day" and tokens[i - 1] == "##s":
                pass
            elif i > 0 and tokens[i - 1] == "year":
                pass
            else:
                tokens.pop(i)
                if i + 1 < len(tokens) and tokens[i] == "on" and tokens[i + 1] == ",":
                    tokens.pop(i)
                    tokens.pop(i)
                elif i < len(tokens) and tokens[i] == ",":
                    tokens.pop(i)
                elif i + 1 < len(tokens) and tokens[i] == "that" and tokens[i + 1] == "day":
                    tokens.pop(i)
                    tokens.pop(i)
                elif i + 2 < len(tokens) and tokens[i] == "in" and tokens[i + 1] == "his" and tokens[i + 2] == "life":
                    tokens.pop(i)
                    tokens.pop(i)
                    tokens.pop(i)
                i = i - 1
                changed = True
        i = i + 1
    return changed

def now_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "now":
            if i + 1 < len(tokens):
                if tokens[i + 1] == ",":
                    tokens.pop(i)
                    tokens.pop(i)
                    changed = True
                    i = i - 1
                elif tokens[i + 1] == ".":
                    tokens.pop(i)
                    changed = True
                    i = i - 1
        i = i + 1
    return changed

def finally_filter(tokens):
    changed = False
    i = 0
    while i < len(tokens):
        if tokens[i] == "finally":
            tokens.pop(i)
            changed = True
            if i < len(tokens) and tokens[i] == ",":
                tokens.pop(i)
            i = i - 1
        i = i + 1
    return changed

filters = [instead_filter, ever_filter, anymore_filter, too_filter, eventually_filter, immediately_filter, anyway_filter, soon_filter, later_filter, now_filter, finally_filter]

# Initialization

In [13]:
from transformers import BertTokenizer
import csv

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking')

source_file = open(current_directory + 'cloze_test.csv', 'r', encoding='utf-8')
modified_file = open(current_directory + 'cloze_test_modified.csv', 'w', encoding='utf-8')
triggers_file = open(current_directory + 'cloze_test_triggers.csv', 'w', encoding='utf-8')
triggers_removed_file = open(current_directory + 'cloze_test_triggers_removed.csv', 'w', encoding='utf-8')

source_reader = csv.reader(source_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
modified_writer = csv.writer(modified_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
triggers_writer = csv.writer(triggers_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)
triggers_removed_writer = csv.writer(triggers_removed_file, quotechar='"', delimiter=',', quoting=csv.QUOTE_ALL, skipinitialspace=True)

# Modification

In [14]:
source_lines = []
for source_line in source_reader:
    source_lines.append(source_line)
modified_writer.writerow(source_lines[0])
triggers_writer.writerow(source_lines[0])
triggers_removed_writer.writerow(source_lines[0])
source_lines.pop(0)

for source_line in source_lines:
    changed = False
    tokens1 = tokenizer.tokenize(source_line[-3])
    tokens2 = tokenizer.tokenize(source_line[-2])
    for filter in filters:
        changed = changed or filter(tokens1) or filter(tokens2)
    sentence1 = tokenizer.convert_tokens_to_string(tokens1)
    sentence2 = tokenizer.convert_tokens_to_string(tokens2)
    modified_line = source_line[:-3] + [sentence1] + [sentence2] + source_line[-1:]
    modified_writer.writerow(modified_line)
    if changed:
        triggers_writer.writerow(source_line)
        triggers_removed_writer.writerow(modified_line)

# Wrap-up

In [16]:
source_file.close()
modified_file.close()
triggers_file.close()
triggers_removed_file.close()